In [1]:
import bw2data, bw2calc, bw2io
from edges import EdgeLCIA, get_available_methods
from bw2analyzer.contribution import ContributionAnalysis

/opt/homebrew/Caskroom/miniforge/base/envs/bw25/lib/python3.11/site-packages/bw2calc/__init__.py:50: UserWarning: 
It seems like you have an ARM architecture, but haven't installed scikit-umfpack:

    https://pypi.org/project/scikit-umfpack/

Installing it could give you much faster calculations.

  warnings.warn(UMFPACK_WARNING)


15:01:16 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


In [2]:
bw2data.projects.set_current("bw25_ei310")

In [3]:
bw2data.databases

Databases dictionary with 4 object(s):
	biosphere
	ecoinvent-3.10-cutoff
	h2
	h2_pem

In [4]:
act = bw2data.Database("ecoinvent-3.10-cutoff").random()
#act = [a for a in bw2data.Database("ecoinvent-3.10-cutoff") if "NMC" in #a["name"]][0]
act

'nutrient supply from vinasse, from fermentation of sugar beet molasses' (kilogram, GLO, None)

In [4]:
method = ('GeoPolRisk', '2024')
# method = ('AWARE 1.2c', 'Country', 'non', 'irri', 'yearly')
LCA = EdgeLCIA({act: 1}, method)
LCA.lci()
LCA.lcia()
LCA.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw25/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.21e+13)
  warnings.warn(msg, UmfpackWarning)


9 supplying technosphere flows have not been characterized:
+----------------------------------------+----------------------------+----------+------------------------+
|                  Name                  |     Reference product      | Location |         Amount         |
+----------------------------------------+----------------------------+----------+------------------------+
| petroleum and gas production, offshore | natural gas, high pressure |    TM    |  0.002662842912460651  |
| petroleum and gas production, onshore  | natural gas, high pressure |    TM    | 0.00032271797041581523 |
| petroleum and gas production, onshore  | natural gas, high pressure |    UZ    | 5.289520181088132e-06  |
| petroleum and gas production, offshore | natural gas, high pressure |    TM    | 0.00011088102769920026 |
| petroleum and gas production, onshore  | natural gas, high pressure |    TM    | 3.326541029388725e-05  |
| petroleum and gas production, onshore  | natural gas, high pressure |    T

0.11775190617328551

In [5]:
LCA.ignored_locations

set()

In [5]:
LCA.generate_cf_table()

Matrix type: technosphere


,supplier name,supplier reference product,supplier location,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"petroleum and gas production, onshore","natural gas, high pressure",EC,"market for natural gas, high pressure","natural gas, high pressure",EC,4.645755e-05,0.042343,1.967156e-06
1,"petroleum and gas production, onshore","natural gas, high pressure",EC,"natural gas, burned in gas turbine","natural gas, burned in gas turbine",EC,7.470712e-07,0.042343,3.163330e-08
2,"hard coal, import from ZA",hard coal,IN,market for hard coal,hard coal,IN,1.705755e-02,0.007686,1.311018e-04
3,"petroleum and gas production, offshore","natural gas, high pressure",MX,"natural gas, high pressure, domestic supply wi...","natural gas, high pressure",MX,7.945433e-03,0.059446,4.723204e-04
4,"petroleum and gas production, offshore","natural gas, high pressure",MX,"natural gas, high pressure, import from MX","natural gas, high pressure",US,1.480993e-05,0.049182,7.283885e-07
...,...,...,...,...,...,...,...,...,...
612,"petroleum and gas production, onshore",petroleum,RU,market for petroleum,petroleum,RNA,6.911431e-04,0.034996,2.418727e-05
613,"petroleum and gas production, onshore",petroleum,RU,market for petroleum,petroleum,Europe without Switzerland,9.083625e-03,0.039258,3.566090e-04
614,"petroleum and gas production, onshore",petroleum,RU,market for petroleum,petroleum,RoW,2.783406e-02,0.026066,7.255100e-04
615,"petroleum and gas production, onshore",petroleum,RU,market for petroleum,petroleum,PE,2.031759e-05,0.015157,3.079474e-07


In [7]:
LCA.ignored_locations

set()

In [4]:
method = ('AWARE 1.2c', 'Country', 'mixed', 'yearly')
convmethod = ('ecoinvent-3.10',
  'EF v3.1 EN15804',
  'water use',
  'user deprivation potential (deprivation-weighted water consumption)')

In [5]:
import bw2calc

convLCA = bw2calc.LCA({act: 1}, convmethod)
convLCA.lci()
convLCA.lcia()
convLCA.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw25/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.21e+13)
  warnings.warn(msg, UmfpackWarning)


0.005048546511965705

In [6]:
LCA = EdgeLCIA({act: 1}, method)
LCA.lci()
LCA.lcia()
LCA.score

Required supplier fields: {'categories', 'name'}
Required consumer fields: {'location', 'classifications'}


TypeError: unhashable type: 'list'

In [48]:
LCA.generate_cf_table()

TypeError: 'NoneType' object is not subscriptable

In [7]:
LCA.ignored_locations

set()

In [19]:
(
    convLCA.characterization_matrix.T
    * convLCA.biosphere_matrix
    * convLCA.demand_array
).sum()

0.02468883093006302

In [20]:
convLCA.demand_array.sum()

1.0

In [21]:
(convLCA.characterization_matrix.T
    * convLCA.biosphere_matrix).sum()

2479894070.894007

In [22]:
convLCA.characterization_matrix.T

<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 4 stored elements and shape (2648, 2648)>

In [23]:
convLCA.biosphere_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 449622 stored elements and shape (2648, 23523)>

In [28]:
LCA.biosphere_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 449622 stored elements and shape (2648, 23523)>

In [45]:
LCA.biosphere_matrix[:, 19955]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 35 stored elements and shape (2648, 1)>

In [46]:
LCA.characterization_matrix[:, 19955]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2 stored elements and shape (2648, 1)>

In [47]:
import numpy as np
np.argwhere(LCA.characterization_matrix[:, 19955])

array([[ 615,    0],
       [1800,    0]], dtype=int32)

In [48]:
LCA.characterization_matrix[615, 19955]

1.245929289

In [49]:
LCA.characterization_matrix[1800, 19955]

-1.245929289

In [50]:
LCA.biosphere_matrix[615, 19955]

0.04682272672653198

In [51]:
LCA.biosphere_matrix[1800, 19955]

0.04765244573354721

-0.001033771212480268

In [60]:
(
    LCA.characterization_matrix[:, 19955].T
    * LCA.biosphere_matrix
    * LCA.demand_array
).sum()

-0.001033771212480268

In [34]:
LCA.characterization_matrix.shape

(2648, 23523)

In [36]:
LCA.characterization_matrix[:, 19955]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2 stored elements and shape (2648, 1)>

In [29]:
LCA.biosphere_matrix.shape

(2648, 23523)

In [30]:
LCA.characterization_matrix.shape

(2648, 23523)

In [37]:
LCA.demand_array.shape

(23523,)

In [42]:
import numpy as np
np.argwhere(LCA.demand_array)

array([[19955]])

In [43]:
import numpy as np
np.argwhere(convLCA.demand_array)

array([[19955]])

In [32]:
convLCA.characterized_inventory[:, 19955].todense().sum()

0.024689276655995802

In [31]:
LCA.characterized_inventory[:, 19955].todense().sum()

-0.0010337898759252631

In [44]:
(LCA.characterized_inventory * LCA.demand_array).sum()

ValueError: dimension mismatch

In [27]:
LCA.biosphere_matrix.sum() == convLCA.biosphere_matrix.sum()

True

In [16]:
LCA.demand_array.sum()

1.0

In [10]:
ContributionAnalysis().annotated_top_emissions(LCA, limit=10)

[(-0.7163026759599563,
  0.2935884512561416,
  'Water' (cubic meter, None, ('water',))),
 (0.6688484258710877,
  0.23508477974056977,
  'Water, turbine use, unspecified natural origin' (cubic meter, None, ('natural resource', 'in water'))),
 (0.04781804107790834,
  0.055576536835671285,
  'Water, cooling, unspecified natural origin' (cubic meter, None, ('natural resource', 'in water'))),
 (0.00020712443746158797,
  0.0014826398294574454,
  'Water, river' (cubic meter, None, ('natural resource', 'in water'))),
 (0.00013507704946424606,
  2.457000304668285e-05,
  'Water, unspecified natural origin' (cubic meter, None, ('natural resource', 'in water'))),
 (9.184280114282116e-05,
  0.0038333509602456743,
  'Water, well, in ground' (cubic meter, None, ('natural resource', 'in water'))),
 (-3.826499647325903e-05,
  6.488861959737442e-05,
  'Water' (cubic meter, None, ('water', 'surface water'))),
 (-5.463602033553495e-06,
  1.1898117270160745e-06,
  'Water' (cubic meter, None, ('water', 'gro

In [9]:
get_available_methods()

[('AWARE 1.2c', 'Country', 'irri', 'apr'),
 ('AWARE 1.2c', 'Country', 'irri', 'aug'),
 ('AWARE 1.2c', 'Country', 'irri', 'dec'),
 ('AWARE 1.2c', 'Country', 'irri', 'feb'),
 ('AWARE 1.2c', 'Country', 'irri', 'jan'),
 ('AWARE 1.2c', 'Country', 'irri', 'jul'),
 ('AWARE 1.2c', 'Country', 'irri', 'jun'),
 ('AWARE 1.2c', 'Country', 'irri', 'mar'),
 ('AWARE 1.2c', 'Country', 'irri', 'may'),
 ('AWARE 1.2c', 'Country', 'irri', 'nov'),
 ('AWARE 1.2c', 'Country', 'irri', 'oct'),
 ('AWARE 1.2c', 'Country', 'irri', 'sep'),
 ('AWARE 1.2c', 'Country', 'irri', 'yearly'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'apr'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'aug'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'dec'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'feb'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jan'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jul'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jun'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'mar'),
 ('AWARE 1.2c', 'Country', 'non', 'irr

In [6]:
LCA.generate_cf_table()

,flow,name,reference product,location,amount,CF,impact
0,"(Water, cooling, unspecified natural origin, (...","electricity production, oil","electricity, high voltage",CA-PE,6.246820e-08,1.713382,7.600501e-09
1,"(Water, cooling, unspecified natural origin, (...","electricity production, lignite","electricity, high voltage",MK,8.000828e-05,48.894393,2.669999e-04
2,"(Water, cooling, unspecified natural origin, (...","electricity production, nuclear, boiling water...","electricity, high voltage",TW,4.689918e-04,2.225805,6.656068e-05
3,"(Water, cooling, unspecified natural origin, (...","electricity production, oil","electricity, high voltage",IN-BR,2.416318e-06,14.183279,3.046240e-06
4,"(Water, cooling, unspecified natural origin, (...","phosphoric acid production, dihydrate process","fluosilicic acid, without water, in 22% soluti...",MA,1.076705e-08,87.352520,3.328043e-07
...,...,...,...,...,...,...,...
3859,"(Water, (water, surface water))","treatment of waste polyethylene, municipal inc...",waste polyethylene,CH,6.462038e-07,-0.965402,-1.287748e-10
3860,"(Water, (water, surface water))","petroleum and gas production, onshore",petroleum,QA,1.151298e-04,-49.511931,-9.146944e-08
3861,"(Water, (water, surface water))","petroleum and gas production, onshore","natural gas, high pressure",TH,7.372886e-05,-6.981635,-3.358919e-08
3862,"(Water, (water, surface water))","treatment of waste emulsion paint, municipal i...",waste emulsion paint,CH,9.712645e-09,-0.965402,-5.519384e-11


In [8]:
from constructive_geometries import geomatcher

geo = geomatcher.Geomatcher()
geo.contained("RER")

[('ecoinvent', 'RER'),
 ('ecoinvent', 'Europe without Austria'),
 ('ecoinvent', 'Europe without Switzerland'),
 ('ecoinvent', 'Europe without Switzerland and Austria'),
 ('ecoinvent', 'RER w/o CH+DE'),
 ('ecoinvent', 'Europe, without Russia and Türkiye'),
 ('ecoinvent', 'RER w/o RU'),
 ('ecoinvent', 'Europe without Switzerland and France'),
 ('ecoinvent', 'RER w/o DE+NL+RU'),
 ('ecoinvent', 'RER w/o DE+NL+NO'),
 ('ecoinvent', 'ENTSO-E'),
 ('ecoinvent', 'Europe without NORDEL (NCPA)'),
 ('ecoinvent', 'RER w/o DE+NL+NO+RU'),
 ('ecoinvent', 'RER w/o AT+BE+CH+DE+FR+IT'),
 ('ecoinvent', 'UCTE'),
 ('ecoinvent', 'IAI Area, EU27 & EFTA'),
 ('ecoinvent', 'UCTE without Germany'),
 ('ecoinvent', 'UCTE without France'),
 ('ecoinvent', 'UCTE without Germany and France'),
 ('ecoinvent', 'UN-SEUROPE'),
 ('ecoinvent', 'UN-NEUROPE'),
 'GB',
 ('ecoinvent', 'WEU'),
 ('ecoinvent', 'NORDEL'),
 'SI',
 ('ecoinvent', 'BALTSO'),
 'IT',
 ('ecoinvent', 'Russia (Europe)'),
 'LV',
 'NO',
 'FR',
 'GR',
 'MK',
 ('ec

In [8]:
for ds in bw2data.Database("ecoinvent-3.10-cutoff"):
    pass
print("done")

done


In [10]:
LCA.packages

In [12]:
list(LCA.biosphere_dict.items())

[(124190747510779904, 0),
 (124190747510779905, 1),
 (124190747510779906, 2),
 (124190747510779907, 3),
 (124190747510779908, 4),
 (124190747510779909, 5),
 (124190747510779910, 6),
 (124190747510779911, 7),
 (124190747510779912, 8),
 (124190747510779913, 9),
 (124190747510779914, 10),
 (124190747510779916, 11),
 (124190747510779917, 12),
 (124190747510779919, 13),
 (124190747510779920, 14),
 (124190747510779921, 15),
 (124190747510779922, 16),
 (124190747510779924, 17),
 (124190747510779925, 18),
 (124190747510779929, 19),
 (124190747510779930, 20),
 (124190747510779931, 21),
 (124190747510779935, 22),
 (124190747510779936, 23),
 (124190747510779937, 24),
 (124190747510779938, 25),
 (124190747510779939, 26),
 (124190747510779941, 27),
 (124190747510779942, 28),
 (124190747510779943, 29),
 (124190747510779944, 30),
 (124190747510779945, 31),
 (124190747510779946, 32),
 (124190747510779947, 33),
 (124190747510779948, 34),
 (124190747510779949, 35),
 (124190747510779950, 36),
 (124190747

In [15]:
LCA.technosphere_edges

{(4908, 188),
 (7591, 23451),
 (17364, 13626),
 (21908, 15455),
 (11326, 16251),
 (10199, 6381),
 (2666, 9852),
 (21287, 19686),
 (370, 18526),
 (21926, 196),
 (14652, 17891),
 (15828, 2770),
 (21567, 16293),
 (12878, 12906),
 (15270, 3940),
 (8456, 12760),
 (5029, 5029),
 (9332, 3763),
 (1364, 890),
 (9664, 2058),
 (897, 15010),
 (9739, 14475),
 (10323, 940),
 (9522, 9522),
 (5051, 3098),
 (3652, 3652),
 (13874, 8929),
 (3533, 2240),
 (18909, 18909),
 (13964, 17490),
 (10724, 11837),
 (8050, 15455),
 (6739, 11692),
 (8423, 20167),
 (11229, 23084),
 (11997, 18526),
 (21895, 20481),
 (22046, 2183),
 (139, 284),
 (12931, 6501),
 (21895, 7649),
 (22121, 4020),
 (8378, 14008),
 (4675, 19965),
 (8536, 11184),
 (14802, 14802),
 (12058, 7101),
 (19490, 22045),
 (14044, 21576),
 (1126, 16186),
 (6329, 10481),
 (5176, 14474),
 (7204, 23513),
 (7328, 18853),
 (22439, 5268),
 (14259, 4202),
 (9616, 21615),
 (4725, 3463),
 (15316, 15962),
 (20380, 13131),
 (6441, 7689),
 (22438, 23510),
 (12951, 1

In [12]:
import bw2io
fp = "/Users/romain/GitHub/premise/premise/data/additional_inventories/lci-hydrogen-electrolysis.xlsx"

i = bw2io.ExcelImporter(fp)
i.apply_strategies()


Extracted 1 worksheets in 0.05 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.06 seconds


In [14]:
i.match_database(fields=["name", "reference product", "location"])
i.match_database("ecoinvent-3.10-cutoff", fields=["name", "reference product", "location"])
i.match_database(fields=["name", "reference product", "location"])
i.match_database("biosphere", fields=["name", "categories"])
i.statistics()

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Graph statistics for `h2_pem` importer:
16 graph nodes:
	None: 16
309 graph edges:
	technosphere: 277
	production: 16
	biosphere: 16
305 edges to the following databases:
	ecoinvent-3.10-cutoff: 257
	h2_pem: 32
	biosphere: 16
2 unique unlinked edges (4 total):
	technosphere: 2




(16, 309, 4, 0)

In [15]:
    i.drop_unlinked(i_am_reckless=True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds


In [16]:
i.write_database()

11:49:19 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 16/16 [00:00<00:00, 413.98it/s]

11:49:19 [info     ] Vacuuming database            


Created database: h2_pem


In [19]:
[
    (a["name"], a["reference product"], a["location"], a["code"])
    for a in bw2data.Database("h2_pem") if "PEM" in a["name"]
]

[('electrolyzer production, 1MWe, PEM, Balance of Plant',
  'electrolyzer, 1MWe, PEM, Balance of Plant',
  'RER',
  'f3fda92c6f40b652aee44ac8e68053a4'),
 ('treatment of fuel cell balance of plant, 1MWe, PEM',
  'used fuel cell balance of plant, 1MWe, PEM',
  'RER',
  '937fc767b9547738e849fe73fee8c51f'),
 ('electrolyzer production, 1MWe, PEM, Stack',
  'electrolyzer, 1MWe, PEM, Stack',
  'RER',
  '7b498745a482ad10549d1d4330c7abf0'),
 ('hydrogen production, gaseous, 30 bar, from PEM electrolysis, from grid electricity',
  'hydrogen, gaseous, 30 bar',
  'RER',
  'c864ccf9cb50864488753874b5d69057'),
 ('treatment of fuel cell stack, 1MWe, PEM',
  'used fuel cell stack, 1MWe, PEM',
  'RER',
  'affe261eea51b2ddb28189f84c7c28b4')]

In [22]:
[a["code"] for a in bw2data.Database("ecoinvent-3.10-cutoff") if a["location"] == "FR" and "offshore" in a["name"]]

['7ba2c49140d98d7c941cef85f6f58d75']